In [3]:
from __future__ import absolute_import, division, print_function

import torch
import torchvision.models as models

vgg19 = models.vgg19(pretrained=True)
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.autograd import Variable

from glob import glob
import os

import numpy as np
import pandas as pd
import json
import re

import matplotlib
from matplotlib import pylab, mlab, pyplot
%matplotlib inline
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('talk')
sns.set_style('white')

from PIL import Image
import base64

from embeddings import *

import warnings
warnings.simplefilter("ignore")

!pip install argparse
import argparse

# directory & file hierarchy
proj_dir = os.path.abspath('../../..')
analysis_dir = os.getcwd()
results_dir = os.path.join(proj_dir,'results')
plot_dir = os.path.join(results_dir,'plots')
csv_dir = os.path.join(results_dir,'csv')
exp_dir = os.path.abspath(os.path.join(proj_dir,'experiments'))
sketch_dir = os.path.abspath(os.path.join(proj_dir,'sketches'))

drawings_dir = os.path.join(sketch_dir,'run3_run4')
#h.save_sketches(D_run3_correct, complete_games, sketch_dir, 'run3_run4', iterationNum)

/home/megsano/.local/lib/python2.7/site-packages/cryptography/hazmat/primitives/constant_time.py:26: CryptographyDeprecationWarning: Support for your Python version is deprecated. The next version of cryptography will remove support. Please upgrade to a 2.7.x release that supports hmac.compare_digest as soon as possible.
  utils.DeprecatedIn23,
You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
# retrieve sketch paths
def list_files(path, ext='png'):
    result = [y for x in os.walk(path) for y in glob(os.path.join(x[0], '*.%s' % ext))]
    return result

def check_invalid_sketch(filenames,invalids_path='drawings_to_exclude.txt'):    
    if not os.path.exists(invalids_path):
        print('No file containing invalid paths at {}'.format(invalids_path))
        invalids = []        
    else:
        x = pd.read_csv(invalids_path, header=None)
        x.columns = ['filenames']
        invalids = list(x.filenames.values)
    valids = []   
    basenames = [f.split('/')[-1] for f in filenames]
    for i,f in enumerate(basenames):
        if f not in invalids:
            valids.append(filenames[i])
    return valids

def make_dataframe(Labels):    
    Y = pd.DataFrame([Labels])
    Y = Y.transpose()
    Y.columns = ['label']
    return Y

def normalize(X):
    X = X - X.mean(0)
    X = X / np.maximum(X.std(0), 1e-5)
    return X

def preprocess_features(Features, Y):
    _Y = Y.sort_values(['label'])
    inds = np.array(_Y.index)
    _Features = normalize(Features[inds])
    _Y = _Y.reset_index(drop=True) # reset pandas dataframe index
    return _Features, _Y

def save_features(Features, Y, layer_num, data_type,feat_path='./sketch_features'):
    if not os.path.exists('./sketch_features'):
        os.makedirs('./sketch_features')
    layers = ['P1','P2','P3','P4','P5','FC6','FC7']
    np.save(os.path.join(feat_path,'FEATURES_{}_{}.npy'.format(layers[int(layer_num)], data_type)), Features)
    Y.to_csv(os.path.join(feat_path,'METADATA_{}.csv'.format(data_type)))
    return layers[int(layer_num)]

In [13]:
parser = argparse.ArgumentParser()
parser.add_argument('--data', type=str, help='full path to images', default=drawings_dir)#'combined_final_png_drawings')
parser.add_argument('--layer_ind', help='fc6 = 5, fc7 = 6', default=5)
parser.add_argument('--data_type', help='"images" or "sketch"', default='sketch')
parser.add_argument('--spatial_avg', type=bool, help='collapse over spatial dimensions, preserving channel activation only if true', default=False)     
parser.add_argument('--test', type=bool, help='testing only, do not save features', default=False)  
parser.add_argument('--ext', type=str, help='image extension type (e.g., "png")', default="png")   

args = parser.parse_args(args=[])
# args = parser.parse_args(argv[1:])

## get list of all sketch paths
image_paths = sorted(list_files(args.data,args.ext))
print('Length of image_paths before filtering: {}'.format(len(image_paths)))

## filter out invalid sketches
image_paths = check_invalid_sketch(image_paths)
print('Length of image_paths after filtering: {}'.format(len(image_paths)))    

## extract features
layers = ['P1','P2','P3','P4','P5','FC6','FC7']
extractor = FeatureExtractor(image_paths,layer=args.layer_ind,data_type=args.data_type,spatial_avg=args.spatial_avg)
Features, Labels = extractor.extract_feature_matrix()   

# organize metadata into dataframe
Y = make_dataframe(Labels)
_Features, _Y = preprocess_features(Features, Y)

if args.test==False:
    layer = save_features(_Features, _Y, args.layer_ind, args.data_type) # g,trialNum,target, repetition, iterationNum

Length of image_paths before filtering: 1841
No file containing invalid paths at drawings_to_exclude.txt
Length of image_paths after filtering: 1841
CUDA DEVICE NUM: 0
Batch 1
batch size: 64
Batch 2
batch size: 64
Batch 3
batch size: 64
Batch 4
batch size: 64
Batch 5
batch size: 64
Batch 6
batch size: 64
Batch 7
batch size: 64
Batch 8
batch size: 64
Batch 9
batch size: 64
Batch 10
batch size: 64
Batch 11
batch size: 64
Batch 12
batch size: 64
Batch 13
batch size: 64
Batch 14
batch size: 64
Batch 15
batch size: 64
Batch 16
batch size: 64
Batch 17
batch size: 64
Batch 18
batch size: 64
Batch 19
batch size: 64
Batch 20
batch size: 64
Batch 21
batch size: 64
Batch 22
batch size: 64
Batch 23
batch size: 64
Batch 24
batch size: 64
Batch 25
batch size: 64
Batch 26
batch size: 64
Batch 27
batch size: 64
Batch 28
batch size: 64
Batch 29
batch size: 64
stopped!


##### manually save it 

In [14]:
feat_path = './sketch_features'
layer_num = args.layer_ind
data_type = 'sketch'
layers = ['P1','P2','P3','P4','P5','FC6','FC7']
out_path = os.path.join(feat_path,'FEATURES_{}_{}.npy'.format(layers[int(layer_num)], data_type))
np.save(out_path, _Features)